SDV or Synthetic Data Vault is a Python package to generate synthetic data based on the dataset provided. The generated data could be single-table, multi-table, or time-series, depending on the scheme you provided in the environment. Also, the generated would have the same format properties and statistics as the provided dataset.

SDV generates synthetic data by applying mathematical techniques and machine learning models such as the deep learning model. Even if the data contain multiple data types and missing data, SDV will handle it, so we only need to provide the data (and the metadata when required).

In [3]:
!pip install sdv

     ------------------------------------ 102.1/102.1 kB 734.9 kB/s eta 0:00:00
     -------------------------------------- 132.0/132.0 kB 1.9 MB/s eta 0:00:00
     ---------------------------------------- 61.8/61.8 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 1.6/1.6 MB 295.5 kB/s eta 0:00:00
     -------------------------------------- 53.2/53.2 kB 390.2 kB/s eta 0:00:00
     ------------------------------------ 213.4/213.4 kB 419.8 kB/s eta 0:00:00
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [4]:
import pandas as pd
data = pd.read_csv('malware.csv')
data.head()

,Bot Address,CnC Address,Attack(Y/N),Attacked Address,URLHaus Tags,Collection Date(DD/MM/YYYY),elf,Mirai,upx,bashlite,Gafgyt,Count of Attacked Address
0,192.168.17.134,46.101.127.55,N,0,elf;Mirai,28/07/2020,1,1,0,0,0,1
1,192.168.99.73,45.80.37.166,Y,5.196.202.200,elf;Mirai,15/07/2019,1,1,0,0,0,1
2,192.168.99.73,65.220.202.53,N,0,elf;Mirai;upx,5/11/2019,1,1,1,0,0,1
3,192.168.17.134,179.43.149.13,N,0,bashlite;elf;Gafgyt,17/07/2020,1,0,0,1,1,1
4,192.168.182.145,93.174.93.213,N,0,bashlite;elf;Gafgyt,17/01/2020,1,0,0,1,1,1


In [5]:
from sdv.tabular import GaussianCopula
model = GaussianCopula()
model.fit(data)

c:\Users\user\anaconda3\lib\site-packages\copulas\univariate\truncated_gaussian.py:45: RuntimeWarning: invalid value encountered in double_scalars
  a = (self.min - loc) / scale
c:\Users\user\anaconda3\lib\site-packages\copulas\univariate\truncated_gaussian.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  b = (self.max - loc) / scale


In [12]:
sample = model.sample(200000)
sample.head()

,Bot Address,CnC Address,Attack(Y/N),Attacked Address,URLHaus Tags,Collection Date(DD/MM/YYYY),elf,Mirai,upx,bashlite,Gafgyt,Count of Attacked Address
0,192.168.17.134,198.98.59.176,N,0,elf;Mirai,03/07/2019,1,1,0,0,0,1
1,192.168.11.47,51.38.99.208,N,0,elf;Mirai,15/07/2019,1,0,0,0,1,1
2,192.168.94.85,167.114.194.6,N,0,elf;Mirai,21/09/2020,1,0,0,0,0,2
3,192.168.99.73,193.239.147.66,N,0,elf;Mirai,30/10/2019,1,1,0,0,0,1
4,192.168.161.202,65.220.202.53,N,0,elf;Mirai,17/07/2020,0,1,0,0,0,1


 This evaluation would compare the real dataset with the sample dataset. Many tests are available, but we would only focus on the Kolmogorov–Smirnov (KS) and Chi-Squared (CS) tests.

In [13]:
from sdv.evaluation import evaluate

evaluate(sample, data, metrics=['CSTest','KSTest'], aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,CSTest,Chi-Squared,0.978794,0.978794,0.0,1.0,MAXIMIZE,None


KSTest is used to compare the continuous columns, and CSTest compares the discrete columns. Both tests result in a normalized score between 0 to 1, with the target is to maximize the score. From the result above, we can assess that the discrete sample columns are good (almost similar to the real data). In contrast, continuous columns might have a deviation in distribution.

In [14]:
sample.shape

(200000, 12)

In [15]:
sample.to_csv('manufactured.csv', index=False)